In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
print(X.shape)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

(442, 10)


In [10]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
# loss_fn = nn.L1Loss()
# loss_fn = nn.NLLLoss(reduction='mean')
opt = pytorch_soom.GaussNewton(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.1, line_search_method='backtrack', line_search_cond='armijo')
# opt = pytorch_soom.GaussNewton(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='wolfe')
# opt = pytorch_soom.GaussNewton(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='strong-wolfe')
# opt = pytorch_soom.GaussNewton(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='goldstein')

all_loss = {}
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss[epoch+1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch+1] += loss
    all_loss[epoch+1] /= len(data_loader)
    print(', loss: {}'.format(all_loss[epoch+1].detach().numpy().item()))

epoch:  0, loss: 0.07332880049943924
epoch:  1, loss: 0.07270583510398865
epoch:  2, loss: 0.0721953958272934
epoch:  3, loss: 0.0711686909198761
epoch:  4, loss: 0.060094282031059265
epoch:  5, loss: 0.05767282843589783
epoch:  6, loss: 0.05751418694853783
epoch:  7, loss: 0.05682522431015968
epoch:  8, loss: 0.054821696132421494
epoch:  9, loss: 0.054254718124866486
epoch:  10, loss: 0.0540747232735157
epoch:  11, loss: 0.05399547889828682
epoch:  12, loss: 0.05380211025476456
epoch:  13, loss: 0.053541336208581924
epoch:  14, loss: 0.053426120430231094
epoch:  15, loss: 0.05305492877960205
epoch:  16, loss: 0.05284949392080307
epoch:  17, loss: 0.05276552215218544
epoch:  18, loss: 0.05257149413228035
epoch:  19, loss: 0.05227387696504593
epoch:  20, loss: 0.052146829664707184
epoch:  21, loss: 0.051718421280384064
epoch:  22, loss: 0.051493339240550995
epoch:  23, loss: 0.05140342935919762
epoch:  24, loss: 0.05114687234163284
epoch:  25, loss: 0.05082958936691284
epoch:  26, loss: